In [26]:
# default_exp data.finance

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
from libs import is_lib_exists

In [29]:
# lxml,html5lib required by yfinance
required_libs  = ["numpy", "pandas", "seaborn", "matplotlib", "yfinance", "lxml", "html5lib"]

In [30]:
for lib in required_libs:
    if not is_lib_exists(lib):
        print(f"Installing {lib}")
        !pip install {lib}
    else:
        print(f"{lib} exists")

numpy exists
pandas exists
seaborn exists
matplotlib exists
yfinance exists
lxml exists
html5lib exists


In [39]:
#export
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
from pathlib import Path

In [32]:
%matplotlib inline

# Finance Data

> Functionality to gather finance data.

The classes here provide bases classes used to gather finance data for model training.

In [33]:
#hide
from nbdev.showdoc import *

In [44]:
DATASET_PATH = Path("../../data")
FTSE_KLSE = "ftse_klse.csv"
DATASET = DATASET_PATH/FTSE_KLSE

## FTSE KLSE

In [46]:
df_stocks = pd.read_csv(DATASET, header=0, sep=',', quotechar='"')
df_stocks.head(10)

,S.No.,Company Name,Ticker
0,1,Hong Leong Bank Berhad,5819.KL
1,2,IOI Corp.Bhd,1961.KL
2,3,IHH Healthcare Berhad,5225.KL
3,4,Malayan Banking Berhad,1155.KL
4,5,Hartalega Holdings Berhad,5168.KL
5,6,PPB Group Berhad,4065.KL
6,7,SIMEPLT,5285.KL
7,8,Kuala Lumpur Kepong Berhad,2445.KL
8,9,Public Bank Berhad,1295.KL
9,10,Maxis Berhad,6012.KL


In [51]:
#for ticker in df_stocks['Ticker']:
#    print(ticker)
stock = yf.Ticker(df_stocks['Ticker'][0])
type(stock.dividends)

pandas.core.series.Series